# Unsupervised learning

### The goal of the notebook is to research some models x features sets out of *.pcaps

In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import collections
import seaborn as sns
import pprint

import pyshark
import networkx as nx

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
from sklearn.metrics import pairwise_distances, silhouette_score
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

from itertools import product

# PATH change to access library cyberlib
import sys
sys.path.append('/home/benjamin/Folders_Python/Cyber/libs')
import cyberlib as cbl

In [2]:
# logging set-up

lg = cbl.GetLogger('/home/benjamin/Folders_Python/Cyber/logs/unsupervised_learning.log')
logger = lg.get_custom_logger()

# start your engine
logger.info("-------- new run --------")

### Get the *.pcap to play with

In [3]:
PCAPFILE = '/home/benjamin/Folders_Python/Cyber/data/input_pcaps/bigFlows.pcap'

In [4]:
capture = pyshark.FileCapture(
    input_file=PCAPFILE,
    use_ek=False
)

logger.info(f'-- created a capture object in PyShark with pcap file = {PCAPFILE} --')

In [5]:
pp = pprint.PrettyPrinter(indent=4)

for id in range(5):
    p = cbl.PyPacket(capture[id])
    pp.pprint(p.data)
    print('-------------')

{   'ETH': {   'dst': '00:90:7f:3e:02:d0',
               'src': '14:10:9f:d4:90:db',
               'type': '0x00000800'},
    'IP': {   'dst': '68.64.21.62',
              'flags': '0x00000000',
              'hdr_len': '20',
              'id': '0x0000fd44',
              'len': '1154',
              'proto': '17',
              'src': '172.16.133.57',
              'ttl': '64',
              'version': '4'},
    'TIMESTAMP': {'ts': datetime.datetime(2013, 2, 26, 23, 2, 35, 953172)},
    'UDP': {   'dstport': '1853',
               'length': '1134',
               'payload': 'c5:b8:2b:dd:d0:cc:1a:0f:df:bb:0d:0f:05:70:88:50:11:77:f1:b1:95:e2:ec:3a:88:65:92:aa:26:78:45:11:ac:84:4a:6f:aa:9b:6a:c6:90:ce:5f:58:ea:63:24:53:40:19:30:55:18:eb:ec:25:37:44:42:e8:d9:95:e1:e8:a9:8c:05:18:a4:83:ca:28:74:8b:f5:1d:15:b5:61:16:52:9b:e2:ea:e8:95:d2:37:1e:89:3f:81:2e:d6:d3:8d:23:9d:9f:65:01:1b:05:64:2e:49:72:66:f8:1a:01:49:55:95:c0:a7:07:e8:93:74:1f:cb:b3:1c:11:7e:48:fb:b5:f2:b1:c4:8a:40:31:2b:ef:c7:

### Choice of features

We will look at Ethernet packets part of a TCP conversation

Features at TCP level : source port, destination port, sequence number, acknowledgement number, flags, header length, total length, time delta since last packet, time delta since first packet

Features at IP level : source IP, destination IP, flags, header length, length, identification, ttl, version

Features at ETH level : None

In [6]:
df_raw = cbl.GetTCPDataframeFromFileCapture(filecapture=capture).dataframe

21994it [07:35, 48.24it/s]                   


KeyboardInterrupt: 

In [ ]:
df_raw

### Preprocessing : ordinal encoding, normalization

In [ ]:
# ordinal encoding with Pandas

columns_to_encode_as_ordinal = ['ETH_dst', 'ETH_src', 'IP_id', 'IP_flags', 'IP_src', 'IP_dst']

df_ord = pd.DataFrame()
for c in columns_to_encode_as_ordinal:
    codes, _ = pd.factorize(df_raw[c])
    df_sup = pd.DataFrame(data={ c : list(codes) })
    df_ord = pd.concat([df_ord, df_sup], axis=1)
    
df = df_raw.drop(columns=columns_to_encode_as_ordinal)
df.reset_index(drop=True)

df = pd.concat([df, df_ord], axis=1)

columns_to_drop = ['TIMESTAMP_ts']
df.drop(columns=columns_to_drop, inplace=True)

df

In [ ]:
# the rest of the preprocessing with a scikit learn pipeline

pipe = Pipeline(
    [ ('normalize', StandardScaler() )]
)

In [ ]:
X = df.to_numpy()
X_norm = pipe.fit_transform(X)

In [ ]:
X_norm

In [ ]:
def viz3d(X_embedding, titre='visu 3D'):
    """Utility function to plot a 3D picture

    Args:
        X_embedding (_type_): _description_
    """
    fig = plt.figure(figsize=(6,6))
    ax = plt.axes(projection='3d')
    
    xs = X_embedding[:,0]
    ys = X_embedding[:,1]
    zs = X_embedding[:,2]
    
    ax.scatter3D(xs, ys, zs)
    ax.set_title(titre)
    
    plt.show()
    
    return fig, ax

### Vizualization : PCA, t-SNE

In [ ]:
pca = PCA(
    n_components = 3
)

X_new = pca.fit_transform(X_norm)

expl = [ r*100 for r in pca.explained_variance_ratio_ ]
expl_s = [ f'{e:.1f}%' for e in expl ]
print(f'Variance per principal component : {expl_s}%')

viz3d(X_new, 'PCA')
plt.show()

In [ ]:
X_new = TSNE(
    n_components=3, 
    learning_rate='auto',
    perplexity=10  # reltaed to the number of neighbors used in other manifold learning algorithms. Highly sensitve parameter.
    ).fit_transform(X_norm)

viz3d(X_new, 't-SNE')
plt.show()

### Clustering

k-means

In [ ]:
# finding a good number of clusters

MAX_CLUSTERS = 100
range_clusters = range(2, MAX_CLUSTERS+1)

silhouette = np.zeros(shape=(MAX_CLUSTERS-1))

for n_clusters in range_clusters:
    kmeans = KMeans(
        n_clusters=n_clusters,
        random_state=0,
        init='k-means++', # educated attempt to have a good initialization
        n_init=1  # because k-means++
        )
    labels = kmeans.fit_predict(X_norm)
    silhouette[n_clusters-2] = silhouette_score(X_norm, labels)
    print(f'Silhouette score for {n_clusters} clusters = {silhouette[n_clusters-2]}')
    
n_cluster_opt = np.argmax(silhouette) + 2
print(f'Nb clusters retenu = {n_cluster_opt}')

In [ ]:
fig, ax = plt.subplots()

ax.plot(silhouette)
ax.grid(True)
ax.set_title('Silhouette score v number of kmeans clusters')
plt.show()

In [ ]:
kmeans = KMeans(
    n_clusters=n_cluster_opt,
    random_state=0,
    init='k-means++', # educated attempt to have a good initialization
    n_init=1  # because k-means++
)

In [ ]:
def visu3d(X_norm=X_norm, model=None, labels=None, titre='Visu 3d'):
    """Utility function to plot the 3d PCA projection of the trained model with clusters labels

    Args:
        X_norm (_type_, optional): _description_. Defaults to X_norm.
        model (_type_, optional): _description_. Defaults to None.
        labels (_type_, optional): _description_. Defaults to None.
    """
    
    unique_labels = np.unique(labels)
    
    pca = PCA(n_components = 3)
    X_embedding = pca.fit_transform(X_norm)
    
    fig = plt.figure(figsize=(6,6))
    ax = plt.axes(projection='3d')

    xs = X_embedding[:,0]
    ys = X_embedding[:,1]
    zs = X_embedding[:,2]

    colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
    c = [ colors[labels[i]] if labels[i] >= 0 else [0,0,0,1] for i in range(len(labels)) ]
        
    ax.scatter3D(xs, ys, zs, color=c)
    n_clusters = len(set(unique_labels)) - (1 if -1 in labels else 0)
    titre = titre + f' {n_clusters} clusters'
    ax.set_title(titre)
        
    plt.show()
    
    return fig, ax

In [ ]:
labels = kmeans.fit_predict(X_norm)

fig, ax = visu3d(
    X_norm=X_norm,
    model=kmeans,
    labels=kmeans.fit_predict(X_norm),
    titre=f'kmeans'
)

DBSCAN

In [ ]:
# finding a good pair of Epsilon x Min_samples parameters

# epsilons
distances = pairwise_distances(X_norm, X_norm).flatten() # compute all the euclidian distances between two points in X_norm
dmax = np.max(distances)
print(f'max distance between two points in feature space = {dmax}')
N_EPSILONS = 5
epsilons_range = np.linspace(dmax/100.0, dmax/20.0, num=N_EPSILONS)
# print(epsilons_range)

# minimum samples
n_samples = X_norm.shape[0]
N_MIN_SAMPLES = 10
min_samples_range = np.linspace(n_samples/2000, n_samples/100, num=N_MIN_SAMPLES).astype('int')
# print(min_samples_range)

# silhouettes calculations
silhouette_max = -1.0

for epsilon, min_samples in product(epsilons_range, min_samples_range):
    print(f'-----------------------------------------------------')
    print(f'n_samples = {min_samples}, epsilon = {epsilon}')
    
    db = DBSCAN(
        eps=epsilon,
        min_samples=min_samples
        )
    
    labels = db.fit_predict(X_norm)
    
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)  # number of found clusters (without -1 which is the label for noise
    n_noise_ = list(labels).count(-1)  # number of noisy points
    noise_ratio = n_noise_/n_samples
    
    print(f'number of clusters found is {n_clusters_}')
    print(f'number of noise points found is {n_noise_} out of {n_samples} total (ie {noise_ratio*100:.2f}%)')
    
    if len(set(labels)) >= 2:
        s = silhouette_score(X_norm, labels)
        print(f'Silhouette score for {n_clusters_} clusters = {s:.3f}')
    else:
        print(f'Not calulating silhouette score as only one label found')
    
    if s > silhouette_max:
        silhouette_max = s
        opt_n_clusters = n_clusters_
        opt_eps = epsilon
        opt_min_samples = min_samples
        opt_noise_ratio = noise_ratio
        
print(f'====================================================================================================')
print(f'Best silhouette score is {silhouette_max} with {opt_n_clusters} clusters and {opt_noise_ratio*100:.2f}% noise, for epsilon = {opt_eps} and min_samples = {opt_min_samples}')

In [ ]:
print(f'Running DBSCAN with epsilon = {opt_eps} and min_samples = {opt_min_samples}.\nSilhouette score is {silhouette_max} with {opt_n_clusters} clusters and {opt_noise_ratio*100:.2f}% noise')

db = DBSCAN(
    eps=opt_eps,
    min_samples=opt_min_samples
)

labels = db.fit_predict(X_norm)

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)  # number of found clusters (without -1 which is the label for noise)
n_noise_ = list(labels).count(-1)  # number of noisy points
n_points = X_norm.shape[0]

print(f'=> number of clusters found is {n_clusters_}')
print(f'=> number of noise points found is {n_noise_} out of {n_points} total (ie {n_noise_/n_points*100:.2f}%)')

In [ ]:
labels = db.fit_predict(X_norm)

fig, ax = visu3d(
    X_norm=X_norm,
    model=db,
    labels=db.fit_predict(X_norm),
    titre=f'DBSCAN'
)

### Gaussian Mixtures

In [ ]:
N_MAX_GAUSSIANS = 100
ng_bics = []
bics = []

for i in range(1, N_MAX_GAUSSIANS, 5):
    n_gaussians = i+1
    gm = GaussianMixture(
        n_components=n_gaussians,
        covariance_type='full',
        random_state=42
        )
    
    labels = gm.fit_predict(X_norm)
    bic = gm.bic(X_norm)
    print(f'Model with {n_gaussians} gaussian(s) : BIC score = {bic:.0f}')
    
    ng_bics.append(n_gaussians)
    bics.append(bic)
    
opt_n_gaussians = ng_bics[np.argmin(bics)]
print(f'Best BIC score with {opt_n_gaussians}')

In [ ]:
opt_n_gaussians = ng_bics[np.argmin(bics)]
print(f'Best BIC score {np.min(bics):.0f} with {opt_n_gaussians} gaussians')

fig, ax = plt.subplots()

ax.plot(ng_bics, bics)
ax.grid(True)
ax.set_title(f'BIC score v number of Gaussians in GMM model')
plt.show()

In [ ]:
gm = GaussianMixture(
    n_components=opt_n_gaussians,
    covariance_type='full',
    random_state=42
)

labels = gm.fit_predict(X_norm)

print(f'BIC score = {gm.bic(X_norm):.2f}')

In [ ]:
fig, ax = visu3d(
    X_norm=X_norm,
    model=gm,
    labels=labels,
    titre=f'GMM'
)